# Latitude, Longitude for any pixel in a GeoTiff File
How to generate the latitude and longitude for a pixel at any given position in a GeoTiff file.

In [2]:
from osgeo import ogr, osr, gdal

In [3]:
# opening the geotiff file
ds = gdal.Open('G:\BTP\Satellite\Data\Test2\LE07_L1GT_147040_20050506_20170116_01_T2\LE07_L1GT_147040_20050506_20170116_01_T2_B1.TIF')

In [4]:
col, row, band = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
print(col, row, band)

(7801, 7071, 1)


In [8]:
xoff, a, b, yoff, d, e = ds.GetGeoTransform()
print(xoff, a, b, yoff, d, e)

# details about the params: GDAL affine transform parameters
# xoff,yoff = left corner 
# a,e = weight,height of pixels
# b,d = rotation of the image (zero if image is north up)

(514485.0, 30.0, 0.0, 3300915.0, 0.0, -30.0)


In [9]:
def pixel2coord(x, y):
    """Returns global coordinates from coordinates x,y of the pixel"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [10]:
x,y = pixel2coord(col/2,row/2)
print (x, y)

(631485.0, 3194865.0)


#### These global coordinates are in a *projected coordinated system*, which is a representation of the spheroidal earth's surface, but flattened and distorted onto a plane.
#### To convert these into latitude and longitude, we need to convert these coordinates into *geographic coordinate system*.

In [11]:
# get the existing coordinate system
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(ds.GetProjectionRef())

# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

In [12]:
# converting into geographic coordinate system
lonx, latx, z = transform.TransformPoint(x,y)

In [13]:
print (latx, lonx, z)

(28.874828865052745, 76.34826151423796, 0.0)


In [10]:
# rb = ds.GetRasterBand(1)
px,py = col/2,row/2                   # the pixel location
pix = ds.ReadAsArray(px,py,1,1)     
print pix[0][0]                        # pixel value

11647


# Reverse Geocoding
Converting a lat/long to a physical address or location. 

We want the name of the DISTRICT.

## --------------------------------------------------------------------------------
### API 1: Not so accurate
## --------------------------------------------------------------------------------

In [11]:
coordinates = (latx,lonx)
results = rg.search(coordinates)
print results
print type(results)
print type(results[0])
results[0]

Loading formatted geocoded file...
[{'name': 'Jaisalmer', 'cc': 'IN', 'lon': '70.90387', 'admin1': 'Rajasthan', 'admin2': 'Jaisalmer', 'lat': '26.91763'}]
<type 'list'>
<type 'dict'>


{'admin1': 'Rajasthan',
 'admin2': 'Jaisalmer',
 'cc': 'IN',
 'lat': '26.91763',
 'lon': '70.90387',
 'name': 'Jaisalmer'}

In [12]:
k = 4  # If we want k*k pixels in total from the image


for i in range(0,col,col/k):
    for j in range(0,row,row/k):
        
        # fetching the lat and lon coordinates 
        x,y = pixel2coord(i,j)
        lonx, latx, z = transform.TransformPoint(x,y)
        
        # fetching the name of district
        coordinates = (latx,lonx)
        results = rg.search(coordinates)
        
        # The pixel value for that location
        px,py = i,j                   
        pix = ds.ReadAsArray(px,py,1,1)     
        pix = pix[0][0]
        
        # printing
        s = "The pixel value for the location Lat: {0:5.1f}, Long: {1:5.1f} ({2:15}) is {3:7}".format(latx,lonx,results[0]["name"],pix)
        print (s)

The pixel value for the location Lat:  28.5, Long:  70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:  28.0, Long:  70.2 (Sadiqabad      ) is       0
The pixel value for the location Lat:  27.4, Long:  70.2 (Khairpur       ) is       0
The pixel value for the location Lat:  26.9, Long:  70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:  26.4, Long:  70.2 (Jaisalmer      ) is       0
The pixel value for the location Lat:  28.5, Long:  70.8 (Khanpur        ) is       0
The pixel value for the location Lat:  28.0, Long:  70.8 (Kot Samaba     ) is   11992
The pixel value for the location Lat:  27.4, Long:  70.8 (Jaisalmer      ) is   11661
The pixel value for the location Lat:  26.9, Long:  70.8 (Jaisalmer      ) is   11701
The pixel value for the location Lat:  26.4, Long:  70.8 (Jaisalmer      ) is       0
The pixel value for the location Lat:  28.5, Long:  71.4 (Ahmadpur East  ) is       0
The pixel value for the location Lat:  27.9, Long:  71

## --------------------------------------------------------------------------------
### API 2
## --------------------------------------------------------------------------------

In [13]:
g = geocoder.google([latx,lonx], method='reverse')
print type(g)
print g
print g.city
print g.state
print g.state_long
print g.country
print g.country_long
print g.address

<class 'geocoder.google_reverse.GoogleReverse'>
<[OK] Google - Reverse #6 results>
Amrit Nagar
RJ
Rajasthan
IN
India
Unnamed Road, Amrit Nagar, Rajasthan 342023, India


###### The above wrapper for Google API is not good enough for us. Its not providing us with the district.

#####  Lets try another python library available for the Google Geo API 

In [14]:
results = Geocoder.reverse_geocode(latx, lonx)
print results.city
print results.country
print results.street_address
print results.administrative_area_level_1
print results.administrative_area_level_2  ## THIS GIVES THE DISTRICT !! <----------------
print results.administrative_area_level_3

Amrit Nagar
India
None
Rajasthan
Barmer
None


##### This is what we need, we are getting the district name for given lat,lon coordinates

In [15]:
## Converting the unicode string to ascii string
v = results.country
print type(v)
v = v.encode("ascii")
print type(v)
print v

<type 'unicode'>
<type 'str'>
India


##### Now lets check for an image from Rajasthan

In [16]:
k = 4  # If we want k*k pixels in total from the image


for i in range(0,col,col/k):
    for j in range(0,row,row/k):
        
        # fetching the lat and lon coordinates 
        x,y = pixel2coord(i,j)
        lonx, latx, z = transform.TransformPoint(x,y)
        
        # fetching the name of district
        results = Geocoder.reverse_geocode(latx, lonx)
        
        # The pixel value for that location
        px,py = i,j                   
        pix = ds.ReadAsArray(px,py,1,1)     
        pix = pix[0][0]
        
        # printing
        if results.country.encode('ascii') == 'India':
            s = "Lat: {0:5.1f}, Long: {1:5.1f}, District: {2:12}, Pixel Val: {3:7}".format(latx,lonx,results.administrative_area_level_2,pix)
            print (s)

Lat:  27.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.9, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.4, Long:  70.8, District: Jaisalmer   , Pixel Val:   11661
Lat:  26.9, Long:  70.8, District: Jaisalmer   , Pixel Val:   11701
Lat:  26.4, Long:  70.8, District: Jaisalmer   , Pixel Val:       0
Lat:  27.4, Long:  71.4, District: Jaisalmer   , Pixel Val:   11653
Lat:  26.9, Long:  71.3, District: Jaisalmer   , Pixel Val:   11307
Lat:  26.4, Long:  71.3, District: Barmer      , Pixel Val:       0
Lat:  27.9, Long:  71.9, District: Jaisalmer   , Pixel Val:   11547
Lat:  27.4, Long:  71.9, District: Jodhpur     , Pixel Val:   11637
Lat:  26.9, Long:  71.9, District: Jaisalmer   , Pixel Val:   11881
Lat:  26.4, Long:  71.9, District: Barmer      , Pixel Val:       0
Lat:  28.4, Long:  72.5, District: Bikaner     , Pixel Val:       0
Lat:  27.9, Long:  72.5, District: Bikaner     ,

# Bing Maps REST API

In [17]:
import requests # To make the REST API Call
import json

In [18]:
(latx,lonx)

(26.362894876071923, 72.46299451955348)

In [19]:
url = "http://dev.virtualearth.net/REST/v1/Locations/"
point = str(latx)+","+str(lonx)
key = "Aktjg1X8bLQ_KhLQbVueYMhXDEMo7OaTweIkBvFojInYE4tVxoTp1bGKWbtU_OPJ"
response = requests.get(url+point+"?key="+key)
print(response.status_code)

In [22]:
data = response.json()
print(type(data))

<type 'dict'>


In [23]:
data

{u'authenticationResultCode': u'ValidCredentials',
 u'brandLogoUri': u'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 u'copyright': u'Copyright \xa9 2017 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 u'resourceSets': [{u'estimatedTotal': 1,
   u'resources': [{u'__type': u'Location:http://schemas.microsoft.com/search/local/ws/rest/v1',
     u'address': {u'adminDistrict': u'RJ',
      u'adminDistrict2': u'Jodhpur',
      u'countryRegion': u'India',
      u'formattedAddress': u'Shergarh, RJ, India',
      u'locality': u'Shergarh',
      u'postalCode': u'342028'},
     u'bbox': [26.18815, 72.02842, 26.8617, 72.77681],
     u'confidence': u'Medium',
     u'entityType': u'PopulatedPlace',
     u'geocodePoints': [{u'calculationMethod': u'Rooftop',
       u'coordinates': [26.46525, 72.40963],
    

In [24]:
s = data["resourceSets"][0]["resources"][0]["address"]["adminDistrict2"]
s = s.encode("ascii")
s

'Jodhpur'

In [25]:
url = "http://dev.virtualearth.net/REST/v1/Locations/"
key = "Aktjg1X8bLQ_KhLQbVueYMhXDEMo7OaTweIkBvFojInYE4tVxoTp1bGKWbtU_OPJ"

## Bing API Test
#### For 100 pixel locations

In [33]:
k = 10  # If we want k*k pixels in total from the image


for i in range(0,col,col/k):
    for j in range(0,row,row/k):
        
        ############### fetching the lat and lon coordinates #######################################
        x,y = pixel2coord(i,j)
        lonx, latx, z = transform.TransformPoint(x,y)
        
        ############### fetching the name of district ##############################################
        point = str(latx)+","+str(lonx)
        response = requests.get(url+point+"?key="+key)
        data = response.json()
        s = data["resourceSets"][0]["resources"][0]["address"]
        if s["countryRegion"].encode("ascii") != "India":
            print ("Outside Indian Territory")
            continue
        district = s["adminDistrict2"].encode("ascii")
        
        ############### The pixel value for that location ##########################################
        px,py = i,j                   
        pix = ds.ReadAsArray(px,py,1,1)     
        pix = pix[0][0]
        
        # printing
        s = "Lat: {0:5.1f}, Long: {1:5.1f}, District: {2:12}, Pixel Val: {3:7}".format(latx,lonx,district,pix)
        print (s)

Outside Indian Territory
Outside Indian Territory
Outside Indian Territory
Lat:  27.9, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.7, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.2, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.0, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.8, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.6, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Outside Indian Territory
Outside Indian Territory
Outside Indian Territory
Lat:  27.9, Long:  70.4, District: Jaisalmer   , Pixel Val:       0
Lat:  27.6, Long:  70.4, District: Jaisalmer   , Pixel Val:   11591
Lat:  27.4, Long:  70.4, District: Jaisalmer   , Pixel Val:   11860
Lat:  27.2, Long:  70.4, District: Jaisalmer   , Pixel Val:   12666
Lat:  27.0, Long:  70.4, District:

# We have another player in the ground! 
Can Reverse Geocode by using the python libraries `shapely` and `fiona` with a shapefile for all the district boundaries of India 

In [11]:
import fiona
from shapely.geometry import Point, shape

# Change this for Win7
base = "/Users/macbook/Documents/BTP/Satellite/Data/Maps/Districts/Census_2011"
fc = fiona.open(base+"/2011_Dist.shp")

In [12]:
def reverse_geocode(pt):
    for feature in fc:
        if shape(feature['geometry']).contains(pt):
            return feature['properties']['DISTRICT']
    return "NRI"

In [14]:
k = 10  # If we want k*k pixels in total from the image


for i in range(0,col,col/k):
    for j in range(0,row,row/k):
        
        ############### fetching the lat and lon coordinates #######################################
        x,y = pixel2coord(i,j)
        lonx, latx, z = transform.TransformPoint(x,y)
        
        ############### fetching the name of district ##############################################
        point = Point(lonx,latx)
        district = reverse_geocode(point)
        if district=="NRI":
            print ("Outside Indian Territory")
            continue
        
        ############### The pixel value for that location ##########################################
        px,py = i,j                   
        pix = ds.ReadAsArray(px,py,1,1)     
        pix = pix[0][0]
        
        # printing
        s = "Lat: {0:5.1f}, Long: {1:5.1f}, District: {2:12}, Pixel Val: {3:7}".format(latx,lonx,district,pix)
        print (s)

Outside Indian Territory
Outside Indian Territory
Outside Indian Territory
Lat:  27.9, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.7, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.2, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  27.0, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.8, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.6, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Lat:  26.4, Long:  70.2, District: Jaisalmer   , Pixel Val:       0
Outside Indian Territory
Outside Indian Territory
Outside Indian Territory
Lat:  27.9, Long:  70.4, District: Jaisalmer   , Pixel Val:       0
Lat:  27.6, Long:  70.4, District: Jaisalmer   , Pixel Val:   11591
Lat:  27.4, Long:  70.4, District: Jaisalmer   , Pixel Val:   11860
Lat:  27.2, Long:  70.4, District: Jaisalmer   , Pixel Val:   12666
Lat:  27.0, Long:  70.4, District:

# Now we can proceed to GenFeatures Notebook